In [1]:
import numpy as np
import random
import time
import operator
from collections import defaultdict
from genetic_algorithm import ga

In [2]:
# Set general parameters
chromosome_length = 100
population_size = 10
#best_score_progress = [] # Tracks progress

run_dict = {}
reliable_num = 0
run_num_gen = defaultdict(list)
run_num_fitnessfunc = defaultdict(list)
run_cpu_time = defaultdict(list)

while(reliable_num < 24):
    population_size_array = np.array([])
    outcome_array = np.array([]).reshape(0,2)
    new_population_size = 10
    crossover_operator = '2X'
    bisection_search = 0
    success_population_sizes = np.array([])
    num_generation = 0 
    num_fitnessfunc = 0
    #success_population = np.array([]).reshape(0,100)
    while(True):
        start = time.time()
        population = ga.create_starting_population(int(new_population_size), chromosome_length)
        outcome = ga.run_generation(population, crossover_operator)

        #num_fitnessfunc += 1
        population_size_array = np.hstack((population_size_array,new_population_size))

        if(outcome == "Fail" and new_population_size <= 2560):
            #print(f'Fail - (population size: {len(population)})')
            #num_generation += 1
            if(bisection_search == 0):
                new_population_size = new_population_size*2
            else:
                midpoint_gap = abs((population_size_array[-1] - population_size_array[-2]))/2
                if(midpoint_gap < 10):
                    break
                new_population_size =  population_size_array[-1] + midpoint_gap    

        else:
            #print(len(population),outcome)
            #print(f'Success - (population size: {len(population)})')
            #num_generation += 1
            success_population_sizes = np.hstack((success_population_sizes,new_population_size))
            outcome_array = np.vstack((outcome_array,outcome))
            #success_population = np.vstack((success_population,outcome.reshape(1,100)))
            
            if(len(population_size_array)==1):
                #num_generation, num_fitnessfunc = outcome[0],outcome[1]
                break
                
            bisection_search = 1
            midpoint_gap = abs((population_size_array[-1] - population_size_array[-2]))/2
            
            if(midpoint_gap < 10):
                #num_generation, num_fitnessfunc = outcome[0],outcome[1]
                break
            new_population_size = population_size_array[-1] - midpoint_gap
    #print(outcome_array[-1])        
    end = time.time()
    elapsed_time = end - start
    optimal_population_size = success_population_sizes[-1]
    #optimal_population = success_population[-1] 

    if (str(int(optimal_population_size)) not in run_dict.keys()):
        run_dict[str(int(optimal_population_size))] = [0,0,0,0]
    
        
    run_dict[str(int(optimal_population_size))][0] += 1
    
    run_dict[str(int(optimal_population_size))][1] += outcome_array[-1][0]
    run_num_gen[str(int(optimal_population_size))].append(outcome_array[-1][0])
    
    run_dict[str(int(optimal_population_size))][2] += outcome_array[-1][1]
    run_num_fitnessfunc[str(int(optimal_population_size))].append(outcome_array[-1][1])
    
    run_dict[str(int(optimal_population_size))][3] += elapsed_time
    run_cpu_time[str(int(optimal_population_size))].append(elapsed_time)
    
    run_dict_sorted = dict(sorted(run_dict.items(), key=operator.itemgetter(1),reverse=True))
    reliable_num = run_dict_sorted[(list(run_dict_sorted.keys())[0])][0]
    
#     print(f'\nOptimal population size: {optimal_population_size}')
#     print(f'Number of generations: {outcome}')
    #print(f'Optimal population: {optimal_population}')
    
    
   
    #print(f'Time elapsed: {elapsed_time}')
optimal_population_size = int(list(run_dict_sorted.keys())[0])
observations = run_dict_sorted[list(run_dict_sorted.keys())[0]]
averaged_observations = [i/observations[0] for i in observations[1:]]

f = open("observations.txt", "w")

f.write(f'Chromosome_length: {chromosome_length}\n')
f.write(f'Crossover operator used: {crossover_operator}\n')
f.write(f'Optimal_population_size: {optimal_population_size}\n')
f.write(f'Average number of generations for optimal_population_size: {int(averaged_observations[0])} (std: {np.std(run_num_gen[str(optimal_population_size)])})\n')
f.write(f'Average number of fitness function evaluations for optimal_population_size: {int(averaged_observations[1])} (std: {np.std(run_num_fitnessfunc[str(optimal_population_size)])})\n')
f.write(f'Average CPU time for optimal_population_size: {averaged_observations[2]} (std: {np.std(run_cpu_time[str(optimal_population_size)])})\n')
f.close()